In [1]:
import cobra
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd 

In [2]:
cobra.__version__

'0.26.2'

In [3]:
model = cobra.io.read_sbml_model('models/e_coli/momentiJO1366.xml')
#model = cobra.io.read_sbml_model('eciML1515_batch.xml')
#model = cobra.io.read_sbml_model('ecYeastGEM.xml')

NameError: name 'cobra' is not defined

In [4]:
model.reactions.EX_glc__D_e.lower_bound = -10
#model.reactions.EX_lac__L_e.lower_bound = -10 # This was only used to check diauxie
# This is the standard value used for E. coli, see data from
# https://www.sciencedirect.com/science/article/pii/S2405471216302903?via%3Dihub


In [5]:
for r in model.exchanges:
    if r.lower_bound != 0:
        print(r, r.lower_bound)

EX_ca2_e: ca2_e <=>  -1000.0
EX_cbl1_e: cbl1_e <=>  -0.01
EX_cl_e: cl_e <=>  -1000.0
EX_co2_e: co2_e <=>  -1000.0
EX_cobalt2_e: cobalt2_e <=>  -1000.0
EX_cu2_e: cu2_e <=>  -1000.0
EX_fe2_e: fe2_e <=>  -1000.0
EX_fe3_e: fe3_e <=>  -1000.0
EX_glc__D_e: glc__D_e <=>  -10
EX_h_e: h_e <=>  -1000.0
EX_h2o_e: h2o_e <=>  -1000.0
EX_k_e: k_e <=>  -1000.0
EX_mg2_e: mg2_e <=>  -1000.0
EX_mn2_e: mn2_e <=>  -1000.0
EX_mobd_e: mobd_e <=>  -1000.0
EX_nh4_e: nh4_e <=>  -1000.0
EX_ni2_e: ni2_e <=>  -1000.0
EX_o2_e: o2_e <=>  -1000.0
EX_pi_e: pi_e <=>  -1000.0
EX_so4_e: so4_e <=>  -1000.0
EX_zn2_e: zn2_e <=>  -1000.0


In [6]:
for r in model.exchanges:
    r.upper_bound = 1000


NameError: name 'model' is not defined

# Flux variability analysis

In [47]:
dfv = cobra.flux_analysis.flux_variability_analysis(model, reaction_list = model.exchanges)

In [25]:
dfv2 = cobra.flux_analysis.flux_variability_analysis(model, reaction_list = model.exchanges, fraction_of_optimum = 0.99)

In [49]:
dfv = dfv.loc[dfv.abs().max(1) > 1e-4, :]
dfv.to_csv('dfv.csv', float_format='%.3f', sep = ";")

In [26]:
dfv2 = dfv2.loc[dfv2.abs().max(1) > 1e-4, :]
dfv2.to_csv('dfv2_iecML151_batch.csv', float_format='%.3f', sep = ";")

In [30]:
s = model.optimize()
print(model.summary())

Objective
1.0 BIOMASS_Ec_iJO1366_core_53p95M = 0.7044758867945058

Uptake
------
Metabolite     Reaction      Flux  C-Number  C-Flux
 prot_pool  ER_pool_TG_     0.095         0   0.00%
     ca2_e     EX_ca2_e  0.003667         0   0.00%
      cl_e      EX_cl_e  0.003667         0   0.00%
 cobalt2_e EX_cobalt2_e 1.761E-05         0   0.00%
     cu2_e     EX_cu2_e 0.0004995         0   0.00%
     fe2_e     EX_fe2_e   0.01131         0   0.00%
  glc__D_e  EX_glc__D_e        10         6 100.00%
       k_e       EX_k_e    0.1375         0   0.00%
     mg2_e     EX_mg2_e  0.006111         0   0.00%
     mn2_e     EX_mn2_e 0.0004868         0   0.00%
    mobd_e    EX_mobd_e 9.088E-05         0   0.00%
     nh4_e     EX_nh4_e     7.609         0   0.00%
     ni2_e     EX_ni2_e 0.0002275         0   0.00%
      o2_e      EX_o2_e     19.86         0   0.00%
      pi_e      EX_pi_e    0.6796         0   0.00%
     so4_e     EX_so4_e    0.1777         0   0.00%
     zn2_e     EX_zn2_e 0.0002402  

## Check metabolite turnover

In [65]:
solution = model.optimize()

In [98]:
total_flux_list = []
met_list = ['pyr_c', 'ac_c', 'for_c', 'etoh_c', 'ala__L_c', 'akg_c', 'gln__L_c', 'lys__L_c', 'fum_c', 'dha_c', 'acald_c', 'orot_c', 'ura_c']
for m_id in met_list:
    m = model.metabolites.get_by_id(m_id)
    total_flux = 0
    print("#### {0} #####".format(m_id))
    for r in m.reactions:
        flux_r = solution.fluxes[r.id]*r.get_coefficient(m_id)
        if flux_r > 0:
            total_flux += flux_r
    total_flux_list.append(total_flux)
pd.DataFrame([met_list, total_flux_list])

#### pyr_c #####
#### ac_c #####
#### for_c #####
#### etoh_c #####
#### ala__L_c #####
#### akg_c #####
#### gln__L_c #####
#### lys__L_c #####
#### fum_c #####
#### dha_c #####
#### acald_c #####
#### orot_c #####
#### ura_c #####


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,pyr_c,ac_c,for_c,etoh_c,ala__L_c,akg_c,gln__L_c,lys__L_c,fum_c,dha_c,acald_c,orot_c,ura_c
1,14.503162,4.858798,1.250598,0,0.708929,11.289496,1.275532,0.241749,6.038156,8.979078,1.171939,0.233037,0


In [99]:
model.metabolites.orot_c.summary()

Percent,Flux,Reaction,Definition
100.00%,0.233,DHORD2,dhor__S_c + 0.000230031515597613 prot_pool + q8_c --> orot_c + q8h2_c
Percent,Flux,Reaction,Definition
100.00%,-0.233,ORPT_TG_reverse,orot_c + 0.00309156120831178 prot_pool + prpp_c --> orot5p_c + ppi_c


In [85]:
model.metabolites.dha_c.summary(solution)

Percent,Flux,Reaction,Definition
100.00%,8.979,F6PA_TG_forward,f6p_c + 5.67753828229373e-05 prot_pool --> dha_c + g3p_c
Percent,Flux,Reaction,Definition
100.00%,-8.979,DHAPT,dha_c + pep_c + 0.000456677380424109 prot_pool --> dhap_c + pyr_c


### note to dha
The flux shown here does not represent the standard flux in glycolysis, using 
https://www.genome.jp/entry/2.7.1.121. From a brief google search I found little info about this enzyme in ecoli. The producing reaction https://www.uniprot.org/uniprotkb/P78055/entry is neither the standard flux in E. coli glycolysis, so it is tempting to assume that this one is not necesseraily functioning, at least not at such a high rate

From https://www.sciencedirect.com/science/article/pii/S2405471216302903?via%3Dihub :
__The reactions F6PA and DHAPT are known to be utilized during growth on glycerol but are not active in growth on glucose (Gutknecht et al., 2001).__


In [5]:
with model:
    model.reactions.F6PA_TG_forward.knock_out() 
    # OR KNOCK OUT GENES  b0825 or G_b3946, or 
    # DHAPT (or its genes b2415, G_b1199, G_b1200, G_b1198, G_b2416, 
    # From https://www.sciencedirect.com/science/article/pii/S2405471216302903?via%3Dihub :
    # The reactions F6PA and DHAPT are known to be utilized during growth on glycerol but are not active in growth on glucose (Gutknecht et al., 2001).
    solution_delta_F6PA_TG = model.optimize()
    print(model.metabolites.dha_c.summary(solution_delta_F6PA_TG))
    print(solution_delta_F6PA_TG.objective_value)

NameError: name 'model' is not defined

In [86]:
model.metabolites.acald_c.summary(solution)

Percent,Flux,Reaction,Definition
100.00%,1.172,THRA,0.00121915615627352 prot_pool + thr__L_c --> acald_c + gly_c
Percent,Flux,Reaction,Definition
100.00%,-1.172,ACALD_TG_forward,acald_c + coa_c + nad_c + 0.000842200717739499 prot_pool --> accoa_c + h_c + nadh_c


### Note to acald
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC277611/
Glycine is supposedly made from serine in E. coli

Another paper indicating the glycine is made from glycine, not threonine  https://microbialcellfactories.biomedcentral.com/articles/10.1186/s12934-020-01323-2

Summary of threonine aldolase here:
https://biocyc.org/gene?orgid=ECOLI&id=LTAA-MONOMER

In [95]:
with model:
    model.reactions.THRA.knock_out() 
    model.reactions.THRA2.knock_out() 
    # OR KNOCK OUT GENES  b2551 G_b0870
    solution_delta_THRA = model.optimize()
    print(model.metabolites.acald_c.summary(solution_delta_THRA))
    print(solution_delta_THRA.objective_value)

acald_c
Formula: C2H4O

Producing Reactions
-------------------
Empty DataFrame
Columns: [Percent, Flux, Reaction, Definition]
Index: []

Consuming Reactions
-------------------
Empty DataFrame
Columns: [Percent, Flux, Reaction, Definition]
Index: []
0.6564220253216607


## Check shadow prices
Initial conclusion: shadow prices seem to correspond to the range of FVA values, but not linearly. I excpect (but would need to check) that the spearman correlation between (1/shadow price) and max(FVA) of secreted metabolites is pretty good. 

In [8]:
import scipy.stats as st

In [95]:
model = cobra.io.read_sbml_model('models/e_coli/momentiJO1366.xml')
glucose_uptake = 10
model.reactions.EX_glc__D_e.lower_bound = -glucose_uptake


In [96]:
model.reactions.DHAPT.knock_out()
model.reactions.THRA.knock_out()
model.reactions.THRA2.knock_out()
model.reactions.F6PA_TG_forward.knock_out()


In [97]:
model.optimize()
print(model.summary())

Objective
1.0 BIOMASS_Ec_iJO1366_core_53p95M = 0.6436981589523805

Uptake
------
Metabolite     Reaction      Flux  C-Number  C-Flux
 prot_pool  ER_pool_TG_     0.095         0   0.00%
     ca2_e     EX_ca2_e   0.00335         0   0.00%
      cl_e      EX_cl_e   0.00335         0   0.00%
 cobalt2_e EX_cobalt2_e 1.609E-05         0   0.00%
     cu2_e     EX_cu2_e 0.0004564         0   0.00%
     fe2_e     EX_fe2_e   0.01034         0   0.00%
  glc__D_e  EX_glc__D_e        10         6 100.00%
       k_e       EX_k_e    0.1256         0   0.00%
     mg2_e     EX_mg2_e  0.005584         0   0.00%
     mn2_e     EX_mn2_e 0.0004448         0   0.00%
    mobd_e    EX_mobd_e 8.304E-05         0   0.00%
     nh4_e     EX_nh4_e     6.952         0   0.00%
     ni2_e     EX_ni2_e 0.0002079         0   0.00%
      o2_e      EX_o2_e      17.6         0   0.00%
      pi_e      EX_pi_e    0.6209         0   0.00%
     so4_e     EX_so4_e    0.1623         0   0.00%
     zn2_e     EX_zn2_e 0.0002195  